In [597]:
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import pandas as pd
import umap.umap_ as umap
import plotly.express as px
import numpy as np

In [598]:
d = pd.read_csv(r"doubleExposureData.csv")

In [599]:
data = d.dropna()

allGoodCol = data.columns
allGoodCol = list(allGoodCol)
allGoodCol = allGoodCol[3::]

In [600]:
print("Size is ", len(allGoodCol))
print("Nb of lap of the 1st exposure is", len(data[data["exposure"] == 1]["lap"].unique()))
print("Nb of lap of the 2nd exposure is", len(data[data["exposure"] == 2]["lap"].unique()))

# We can reduce the number of laps in RUN2 to 16
# print("Cropping RUN2 to 16 laps")
# data = data[data['lap'] < len(data[data["exposure"] == 1]["lap"].unique()) + 16]
# print("Number of laps :", len(data["lap"].unique()))

Size is  165
Nb of lap of the 1st exposure is 15
Nb of lap of the 2nd exposure is 31


In [601]:
reducer = umap.UMAP(n_neighbors = 20, 
                    metric = 'cosine', 
                    output_metric = 'euclidean',
                    learning_rate = 1.0, 
                    init = 'spectral', 
                    min_dist = 0.1, 
                    spread = 1.0, 
                    repulsion_strength = 1.0,
                    negative_sample_rate = 5,
                    target_metric = 'categorical', 
                    dens_lambda = 2.0, 
                    dens_frac = 0.3,
                    dens_var_shift=0.1,
                    n_components = 3)

parsedData = data[allGoodCol].values

embedding = reducer.fit_transform(parsedData)
embedding.shape

(3133, 3)

In [602]:
df = pd.DataFrame(embedding, columns=['x1', 'x2', 'x3'])
df["x"] = data.x
df["lap"] = data.lap
df["speed"] = data.speed
df["exposure"] = data.exposure

In [603]:
fig = px.scatter_3d(df, x='x1', y='x2', z='x3',
              color='x', opacity=0.9)

fig.update_traces(marker_size=2)

# tight layout
fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))
fig.show()

In [604]:
fig = px.scatter_3d(df, x='x1', y='x2', z='x3',
              color='lap', opacity=0.9, color_continuous_scale='Bluered_r')

fig.update_traces(marker_size=1.5)

# tight layout
fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))
fig.show()

## Labeled UMAP by lap number

In [605]:
reducer = umap.UMAP(n_neighbors = 20, 
                    metric = 'cosine', 
                    output_metric = 'euclidean',
                    learning_rate = 1.0, 
                    init = 'spectral', 
                    min_dist = 0.1, 
                    spread = 1.0, 
                    repulsion_strength = 1.0,
                    negative_sample_rate = 5,
                    target_metric = 'categorical', 
                    dens_lambda = 2.0, 
                    dens_frac = 0.3,
                    dens_var_shift=0.1,
                    n_components = 3)

parsedData = data[allGoodCol].values

embedding = reducer.fit_transform(parsedData, y = data.lap)
embedding.shape

(3133, 3)

In [606]:
df2 = pd.DataFrame(embedding, columns=['x1', 'x2', 'x3'])
df2["x"] = data.x
df2["lap"] = data.lap
df2["speed"] = data.speed
df2["exposure"] = data.exposure

In [607]:
# We generate a label for the data : the middle point of each lap have the lap number

global_label = ["" for i in range(0, len(df))]

for lap in df2.lap.unique():

    # We get the number of elements in the lap
    nb = len(df2[df2.lap == lap])

    # We get the index of the first point of the global df with that lap
    start = len(df2[df2.lap < lap])

    # We get the middle point
    middle = int(nb/2)

    # We get the coordinates of the middle point in the global dataframe
    middle = start + middle

    # We set the label
    global_label[middle] = str(lap)

df2["label"] = global_label

In [608]:
df2

,x1,x2,x3,x,lap,speed,exposure,label
0,12.863939,21.105997,12.020387,16.917320,1,10.774326,1,
1,13.027982,21.066856,12.030993,18.400847,1,11.275079,1,
2,13.108833,21.056822,12.046865,22.719949,1,11.278806,1,
3,13.190516,21.086021,12.089834,22.018436,1,11.377572,1,
4,13.360104,21.158844,12.153739,23.029066,1,13.119950,1,
...,...,...,...,...,...,...,...,...
3128,-2.654738,3.428916,-3.560328,155.104007,46,29.139482,2,
3129,-2.646618,3.433387,-3.565605,165.005216,46,29.129566,2,
3130,-2.643543,3.430047,-3.561757,172.383165,46,29.358579,2,
3131,-2.590667,3.478672,-3.609306,176.438828,46,28.833411,2,


In [609]:
fig = px.scatter_3d(df2, x='x1', y='x2', z='x3',
              color='x', opacity=0.9, text = "label")

fig.update_traces(marker_size=1.5)
fig.update_traces(textfont_size=10)

# tight layout
fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))
fig.show()

In [610]:
fig = px.scatter_3d(df2, x='x1', y='x2', z='x3',
              color='lap', opacity=0.9, text = "label", color_continuous_scale='Bluered_r')

fig.update_traces(marker_size=1.3)
fig.update_traces(textfont_size=10)

# tight layout
fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))
fig.show()